In [19]:
import pandas as pd
import summary_functions as sf
import arcpy
from arcgis import GeoAccessor
from arcgis.features import FeatureLayer

year_data = 2022
def get_census_data(featureset):
    service_id = {
        'raw_data':'28',
        'summaries':'18',
        'enrollment':'32',
        'visitation':'33',
        'water_use':'34'
    }

    service_number = service_id.get(featureset) 
    service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/'+service_number

    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features

    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])

    return all_data

In [15]:
census_data = get_census_data('raw_data')
tahoe_geography_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'
# Create a FeatureLayer object directly from the REST map service URL
feature_layer = FeatureLayer(tahoe_geography_url)

In [16]:
county_zone = {
    '061':'North Lake',
    '031':'North Lake',
    '005': 'South Lake',
    '017':'South Lake'
}
census_data['Zone'] = census_data['county'].map(county_zone)
#This list is based on input from Kira and Rachel
vehicle_variables = "../Census_Category_Lists/vehicle_categories.csv"
language_variables = "../Census_Category_Lists/Language_Categories.csv"

In [18]:
census_data_yearly = census_data.loc[(census_data['year_sample']==year_data)&(census_data['sample_level']=='block group')]
grouped_vehicles = sf.categorize_values(census_data_yearly, vehicle_variables, 'Broad Category', 'Household Vehicle Numbers: ')
grouped_language = sf.categorize_values(census_data_yearly, language_variables, 'English Fluency', 'English Fluency: ')
combined_grouped = pd.concat([grouped_language,grouped_vehicles])
combined_grouped.to_excel('grouped_2022.xlsx')

c:\Users\amcclary\Documents\GitHub\Demographics\scripts\summary_functions.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_df['value'] = census_df['value'].astype(float)
c:\Users\amcclary\Documents\GitHub\Demographics\scripts\summary_functions.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_df['value'] = census_df['value'].astype(float)


['GEO_ID', 'state', 'county', 'tract', 'block_group', 'variable_category', 'year_sample', 'sample_level', 'dataset', 'census_geom_year', 'TRPAID', 'NEIGHBORHOOD', 'Data_Scale', 'Zone', 'Broad Category']
['GEO_ID', 'state', 'county', 'tract', 'block_group', 'variable_category', 'year_sample', 'sample_level', 'dataset', 'census_geom_year', 'TRPAID', 'NEIGHBORHOOD', 'Data_Scale', 'Zone', 'English Fluency']
